In [ ]:
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests

In [234]:
# ПАРСИНГ

square = []
flats = []
floor = []
floor_counts = []
room = []
metro = []
district = []
zone = []
price = []
metro_foot_minute = []
company = []

floor_regex = r'(\d+/\d+)\s*этаж'# Функция для извлечения этажа из строки
def extract_floor(string):
    match = re.search(floor_regex, string)
    if match:
        return match.group(1)
    else:
        return ''

rooms_regex = r'(\d+)-?комн\.'
def extract_rooms(string):# Функция для извлечения количества комнат из строки
    match = re.search(rooms_regex, string)
    if match:
        return int(match.group(1))
    else:
        return ''

number_regex = r'(\d+[\.,]?\d*)\s*([мm]²)'
def extract_number(string):
    match = re.search(number_regex, string)
    if match:
        return float(match.group(1).replace(',', '.'))
    else:
        return ''
for page in range(1, 50):  # Цикл по номерам страниц
    url = f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={page}&region=1&room1=1&room2=1&room3=1'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        apartments = soup.find_all('div', {'data-name': 'LinkArea'})
        for apartment in apartments:
          div = apartment.find('div', {'data-name': "GeneralInfoSectionRowComponent"})
          if div:
            d = apartment.find('a', {'class': '_93444fe79c--link--VtWj6'}).text
            flats.append(d)
            loc = apartment.find('div', {'class': '_93444fe79c--labels--L8WyJ'}).text
            zone.append(loc.split(', ')[1])
            district_match = re.search(r'(?:р-н|мкр)\s*(\w+(?:\s+\w+)*)(?:\s+р-н|мкр)?', loc)
            if district_match:
              district.append(district_match.group(1))
            else:
              district.append('')
            metro_match = re.search(r'(?:м\.|метро)\s*(\w+(?:\s+\w+)*)', loc)
            if metro_match:
              metro.append(metro_match.group(1))
            else:
              metro.append('')
            t = apartment.find('span', {"data-mark": "MainPrice"})
            if t:
              p = t.text
              p = re.sub(r'\D', '', p)
              price.append(p)
            else:
              price.append('')
            divis = apartment.find('div', {'class': "_93444fe79c--remoteness--q8IXp"})
            if divis is not None:
              distance_text = divis.text.split()[0]
              metro_foot_minute.append(distance_text)
            else:
              metro_foot_minute.append('')
        time.sleep(5)
for flat in flats:
  fl = str(extract_floor(flat))
  r = extract_rooms(flat)
  s = extract_number(flat)
  square.append(s)
  room.append(r)
  if fl !='':
    floor.append(fl.split('/')[0])
    floor_counts.append(fl.split('/')[1])
  else:
    floor.append(fl)
    floor_counts.append(fl)



#for apartment in apartments:
 # div = apartment.find('div', {"data-name": "ContentRow"})
  #if div:
   #   distance_text = div.text
    #  comp = re.findall(r'«(.*?)»', distance_text)
     # company.append(comp)
print(len(square), len(floor), len(floor_counts), len(room), len(metro), len(district), len(zone), len(price), len(metro_foot_minute))

1372 1372 1372 1372 1372 1372 1372 1372 1372


In [235]:
df = pd.DataFrame({
    'room': room,
    'floor': floor,
    'floor_counts': floor_counts,
    'square': square,
    'price': price,
    'metro_foot_minute': metro_foot_minute,
    'zone': zone,
    'district': district,
    'metro': metro,

})
df

,room,floor,floor_counts,square,price,metro_foot_minute,zone,district,metro
0,2,22,34,50.6,59236384,14,ЗАО,Дорогомилово,Кутузовская
1,3,7,10,190.0,139000000,5,ЦАО,Басманный,Бауманская
2,2,2,14,85.6,156750000,2,ЦАО,Якиманка,Боровицкая
3,3,3,15,104.0,65200000,7,САО,Аэропорт,Динамо
4,2,4,13,86.28,101797752,8,ЦАО,Мещанский,Цветной бульвар
...,...,...,...,...,...,...,...,...,...
1367,22,3,19,47.7,32500000,5,ЗАО,Раменки,Аминьевская
1368,3,15,28,63.3,33250000,6,ЗАО,Раменки,Раменки
1369,2,4,11,62.7,33100000,10,ЗАО,Раменки,Раменки
1370,3,3,6,127.0,34300000,11,ТАО (Троицкий),,Крёкшино


In [238]:
df.to_csv('dataset_cian_itog.csv')